In [9]:
from skyfield.api import load, Topos, EarthSatellite, Angle, wgs84
import json
from skyfield.timelib import Time
from datetime import datetime
from scipy.optimize import minimize_scalar
import numpy as np
import pytz
import pandas as pd
import os 

In [19]:
def angular_separation(t):
    """Angular separation (degrees) between Sun and ISS as seen by observer"""
    sun_vec = observer.at(t).observe(sun).apparent()
    iss_vec = observer.at(t).observe(iss).apparent()

    sun_alt, sun_az, sun_dist = sun_vec.altaz()
    sun_alt = sun_alt.degrees
    
    return sun_vec.separation_from(iss_vec).degrees, sun_alt

def convert_t(t):
    """
    Convert a Skyfield Time or a datetime object to Berlin time and format as string.
    """
    if isinstance(t, Time):  # Skyfield Time object
        t_utc = t.utc_datetime()
    elif isinstance(t, datetime):  # regular datetime
        t_utc = t
    else:
        raise TypeError("t must be a Skyfield Time or datetime object")

    t_berlin = t_utc.astimezone(pytz.timezone('Europe/Berlin'))
    return t_berlin.strftime("%Y-%m-%d %H:%M:%S %Z")

In [11]:
ts = load.timescale()

planets = load('de421.bsp')
earth, sun = planets['earth'], planets['sun']

# Define time
t = ts.now()

# Define observation point
observer = earth + wgs84.latlon(53.788419, 9.569346) # Sommerland 
# observer = earth + Topos('52.5245 N', '13.41 E') # Berlin

# Load ISS position data from CelesTrak
max_days = 7.0    # download again once 7 days old
name = 'ISS.csv'  # custom filename, not 'gp.php'

base = 'https://celestrak.org/NORAD/elements/gp.php'
url = base + '?GROUP=stations&FORMAT=json'

if not load.exists(name) or load.days_old(name) >= max_days:
    load.download(url, filename=name)
    print("Updated ISS position")

# Load ISS position
with load.open('ISS.csv', mode='r') as f:
    data = json.load(f) 

# Find the ISS row (using NORAD ID is safest)
iss_row = next(row for row in data if row.get("NORAD_CAT_ID") == 25544)

# Create the EarthSatellite object
iss_geo = EarthSatellite.from_omm(ts, iss_row) # gets the geocentric information on the iss

iss = earth + iss_geo # converts to solar system related positional information

[#################################] 100% ISS.csv


Updated ISS position


### Load planets, define time and observation point

In [21]:
ts = load.timescale()

planets = load('de421.bsp')
earth, sun = planets['earth'], planets['sun']

# Define time
t = ts.now()

# Define observation point
observer = earth + Topos('53.7985 N', '9.54704 E') # Sommerland 
# observer = earth + Topos('52.5245 N', '13.41 E') # Berlin


sun_vec = observer.at(t).observe(sun).apparent()
iss_vec = observer.at(t).observe(iss).apparent()

sun_alt, sun_az, sun_dist = sun_vec.altaz()
iss_alt, iss_az, iss_dist = iss_vec.altaz()

sun_alt = sun_alt.degrees
sun_az = sun_az.degrees

iss_alt = iss_alt.degrees
iss_az = iss_az.degrees

print(f"SUN \nalt:\t{sun_alt:.2f} deg \naz:\t{sun_az:.2f} deg \n")
print(f"ISS \nalt:\t{iss_alt:.2f} deg \naz:\t{iss_az:.2f} deg")

RADIUS_SUN_KM = 695700.0
apparent_diameter = Angle(radians = 2.0 * np.arcsin(RADIUS_SUN_KM / sun_dist.km))

print(f"{apparent_diameter.degrees:.2f} deg")

SUN 
alt:	-36.24 deg 
az:	3.42 deg 

ISS 
alt:	-69.53 deg 
az:	110.00 deg
0.53 deg


In [22]:
print(type(t))

<class 'skyfield.timelib.Time'>


### Cal

In [23]:
# Start time as Skyfield time (not datetime!)
start_date = ts.now()
print("Start date:", convert_t(start_date))

days_to_scan = 30
threshold_deg = 10  # Sun angular diameter ~0.5 deg
candidate_times = []

minutes_per_step = 4  # every 5 minutes

for day_offset in range(days_to_scan):
    day = start_date + day_offset   # Skyfield Time object (not datetime)
    y, m, d, h, mi, s = day.utc     # UTC components

    # Generate times throughout the day
    hours = np.arange(0, 24)
    minutes = np.arange(0, 60, minutes_per_step)
    hh, mm = np.meshgrid(hours, minutes)
    hh = hh.flatten()
    mm = mm.flatten()

    times = ts.utc(y, m, d, hh, mm)  # still Skyfield Time objects

    # Vectorized angular separation check
    for t in times: 
        # print(t.utc_datetime().strftime("%Y-%m-%d %H:%M:%S %Z"))
        separation, sun_alt = angular_separation(t)
        # print(sun_alt)

        if separation <= threshold_deg and sun_alt > 10:
            print(f'{t.utc_datetime()}\t{sun_alt:.2f}\t{separation:.2f}')
            candidate_times.append(t)

Start date: 2025-09-23 01:25:17 CEST
2025-09-25 15:28:00+00:00	14.25	2.27
2025-09-26 14:40:00+00:00	20.15	8.24
2025-10-06 14:32:00+00:00	17.35	9.65
2025-10-06 09:44:00+00:00	28.22	6.81
2025-10-07 13:44:00+00:00	22.24	8.89
2025-10-11 12:04:00+00:00	27.86	7.76
2025-10-11 07:16:00+00:00	12.06	5.39
2025-10-18 09:32:00+00:00	23.32	8.18


In [24]:
print(f"ISS EPOCH: {convert_t(iss_geo.epoch)}\n")
print(f"{'TIME':<25} {'SEPARATION [deg]':<18} {'SUN ALT [deg]':<10}")

results = []

window_minutes = 2
offset_minutes= window_minutes / (24 * 60)

for cand in candidate_times:  # cand is a Skyfield Time object
    start_fine = cand - offset_minutes

    fine_seconds = np.arange(0, (2 * window_minutes * 60) + 1, 1)
    fine_offsets = fine_seconds / 86400.0  # seconds -> days
    times_fine = start_fine + fine_offsets  # Skyfield Time array

    separations = []
    altitudes = []

    for t in times_fine:
        separation, sun_alt = angular_separation(t)
        if separation <= 3:
            separations.append(separation)
            altitudes.append(sun_alt)
    print(len(separations))

    if len(separations) > 0:       
        print("test")
        min_idx = np.argmin(separations)
        best_time = times_fine[min_idx]
        min_sep   = separations[min_idx]
        min_alt = altitudes[min_idx]
        
        results.append({
        "time_utc": best_time.utc_datetime(),
        "separation_deg": min_sep,
        "sun_alt_deg": min_alt
        })

    df = pd.DataFrame(results)
print(df)
        # print(f"{best_time.utc_iso():<25} {min_sep:<18.4f} {sun_alt:<10.4f}")

ISS EPOCH: 2025-09-20 05:40:29 CEST

TIME                      SEPARATION [deg]   SUN ALT [deg]
28
test
0
14
test
0
0
10
test
13
test
0
                   time_utc  separation_deg  sun_alt_deg
0 2025-09-25 15:26:15+00:00        2.099653    14.261309
1 2025-10-06 14:30:06+00:00        1.412119    17.296095
2 2025-10-11 12:02:05+00:00        1.557054    27.869618
3 2025-10-11 07:14:06+00:00        2.285505    12.025662


In [2]:
print(os.path.isfile("/ISS.csv"))

False


transits_20250923.csv


In [6]:
from skyfield.api import wgs84, load, EarthSatellite
from transit import find_transit 
from datetime import datetime

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os
import requests
from utils import convert_t, angular_separation
import numpy as np 
import pandas as pd 

ts = load.timescale()

planets = load('de421.bsp')
earth, sun = planets['earth'], planets['sun']

owner = "NoahJens"
repo = "sun_iss_transit"
file_path = "ISS.csv"
branch = "main"

# Calculate iss data 
url_commit = f"https://api.github.com/repos/{owner}/{repo}/commits?path={file_path}&sha={branch}"
r = requests.get(url_commit)
r.raise_for_status()
latest_commit_sha = r.json()[0]["sha"]

# Fetch CSV at that commit
url_raw_commit = f"https://raw.githubusercontent.com/{owner}/{repo}/{latest_commit_sha}/{file_path}"
r = requests.get(url_raw_commit)
r.raise_for_status()
data = r.json()

# Find the ISS row (using NORAD ID is safest)
iss_row = next(row for row in data if row.get("NORAD_CAT_ID") == 25544)

# Create the EarthSatellite object
iss_geo = EarthSatellite.from_omm(ts, iss_row) # gets the geocentric information on the iss
epoch = convert_t(iss_geo.epoch)

print(epoch)
iss = earth + iss_geo
print(iss)

# Calculate transits 
observer = earth + wgs84.latlon(53.7985, 9.5470)

######################################################
# calc transit 
# Start time as Skyfield time (not datetime!)
start_date = ts.now()
# print("Start date:", convert_t(start_date))

days_to_scan = 7
coarse_threshold_deg = 10  # Sun angular diameter ~0.5 deg
candidate_times = []

minutes_per_step = 4  # every 5 minutes

for day_offset in range(days_to_scan):
    day = start_date + day_offset   # Skyfield Time object (not datetime)
    y, m, d, h, mi, s = day.utc     # UTC components

    # Generate times throughout the day
    hours = np.arange(0, 24)
    minutes = np.arange(0, 60, minutes_per_step)
    hh, mm = np.meshgrid(hours, minutes)
    hh = hh.flatten()
    mm = mm.flatten()

    times = ts.utc(y, m, d, hh, mm)  # still Skyfield Time objects

    # Vectorized angular separation check
    for t in times: 
        print(t.utc_datetime())
        # print(t.utc_datetime().strftime("%Y-%m-%d %H:%M:%S %Z"))
        separation, sun_alt = angular_separation(t, observer, sun, iss)
        # print(sun_alt)

        if separation <= coarse_threshold_deg and sun_alt > 10:
            # print(f'{t.utc_datetime()}\t{sun_alt:.2f}\t{separation:.2f}')
            candidate_times.append(t)

window_minutes = 2
offset_minutes= window_minutes / (24 * 60)
fine_threshold_deg = 2

records = []

for cand in candidate_times:
    start_fine = cand - offset_minutes
    fine_seconds = np.arange(0, (2 * window_minutes * 60) + 1, 1)
    fine_offsets = fine_seconds / 86400.0
    times_fine = start_fine + fine_offsets

    pairs = []
    for t in times_fine:
        separation, sun_alt = angular_separation(t, observer, sun, iss)
        if separation <= fine_threshold_deg:
            pairs.append((t, separation, sun_alt))

    if pairs:
        best_time, min_sep, min_alt = min(pairs, key=lambda x: x[1])
        records.append((convert_t(best_time), min_sep, min_alt))

transit = pd.DataFrame(records, columns=["Time CEST", "Separation [deg]", "Sun altitude [deg]"])
#############################################################
transit["Epoch"] = epoch
transit.to_csv("transits.csv", index=False)

if not transit.empty:

    # File to send
    filename = "transits.csv"
    email_filename = f"transits_{datetime.now().strftime('%Y%m%d')}.csv"

    # Email setup
    msg = MIMEMultipart()
    msg["From"] = os.environ["EMAIL_FROM"]
    msg["To"] = os.environ["EMAIL_TO"]
    msg["Subject"] = "Sun ISS transits"

    # Body
    msg.attach(MIMEText("Please find the CSV attached with a 7 day forecast", "plain"))

    # Attach CSV
    with open(filename, "rb") as f:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(f.read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", f"attachment; filename={email_filename}")
    msg.attach(part)

    # Send
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(os.environ["EMAIL_FROM"], os.environ["EMAIL_PASSWORD"])
        server.send_message(msg)

else:
    print("No transit events — email not sent.")




2022-09-25 00:08:26 CEST
Sum of 3 vectors:
 'de421.bsp' segment 0 SOLAR SYSTEM BARYCENTER -> 3 EARTH BARYCENTER
 'de421.bsp' segment 3 EARTH BARYCENTER -> 399 EARTH
 EarthSatellite 399 EARTH -> ISS (ZARYA) catalog #25544 epoch 2022-09-24 22:08:26 UTC
2025-09-25 00:00:00+00:00


/opt/homebrew/Caskroom/miniconda/base/envs/sun_iss/lib/python3.12/site-packages/jplephem/spk.py:226: RuntimeWarning: invalid value encountered in cast
  index = (index1 + index2 + index3).astype(int)
/opt/homebrew/Caskroom/miniconda/base/envs/sun_iss/lib/python3.12/site-packages/skyfield/curvelib.py:22: RuntimeWarning: invalid value encountered in cast
  i = i.astype(int)


ValueError: light-travel time failed to converge